In [61]:
import uproot as ur
import matplotlib.pyplot as plt
import numpy as np
import awkward as ak
import glob

## Get data, transform ROOT tree into array

In [68]:
energy = 1 # Energy in GeV
file_number = 1 

events = ur.open('root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/pi+\
/{energy}GeV/3to50deg/pi+_{energy}GeV_3to50deg.{file_number:04d}.root:events'.format(energy = energy, file_number = file_number))


root://sci-xrootd.jlab.org//osgpool/eic/ATHENA/RECO/SINGLE/pi+/1GeV/3to50deg/pi+_1GeV_3to50deg.0001.root:events
